In [7]:
from qiskit import *
from qiskit.primitives import *
from qiskit.circuit.library import *
from qiskit.visualization import *
from math import *

In [8]:
DEFAULT_SHOTS = 1024
SEED = 42

In [9]:
def generate_marked_states(k, intList):
    """
    Generate binary strings for integers less than k and present in intList,
    """
    if not intList:
        raise ValueError("intList must not be empty.")
    num_qubits = ceil(log2(max(intList) + 1))
    return [f"{i:0{num_qubits}b}" for i in range(k) if i in intList], num_qubits

In [10]:
def grover_oracle(k, intList):
    """
    Construct a Grover oracle that marks states less than k and present in intList.
    """
    marked_states, num_qubits = generate_marked_states(k, intList)
    qc = QuantumCircuit(num_qubits)

    for target in marked_states:
        zero_inds = [i for i, bit in enumerate(reversed(target)) if bit == '0']
        qc.x(zero_inds)

        if num_qubits > 1:
            qc.h(num_qubits - 1)
            qc.mcx(list(range(num_qubits - 1)), num_qubits - 1)
            qc.h(num_qubits - 1)
        else:
            qc.z(0)

        qc.x(zero_inds)
    return qc

In [11]:
def less_than_k(k, intList):
    """
    Apply Grover's algorithm to find states less than k and in intList. Returns top keys.
    """
    oracle = grover_oracle(k, intList)
    grover_op = GroverOperator(oracle)
    num_qubits = oracle.num_qubits
    markedStateLen = len(generate_marked_states(k, intList)[0])

    optimal_num_iterations = floor(pi / (4 * asin(sqrt(markedStateLen / 2**num_qubits))))
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))
    qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
    qc.measure_all()

    sampler = Sampler()
    sampler.set_options(shots=DEFAULT_SHOTS, seed_simulator=SEED)

    result = sampler.run(qc).result()
    dist = result.quasi_dists
    allSamples = dist[0]
    topSamples = sorted(allSamples.items(), key=lambda x: x[1], reverse=True)[:markedStateLen]
    topKeys = [key for key, value in topSamples]

    return topKeys

In [12]:
k = 7
intList = [4,9,11,14,1,13,6,15]
print(less_than_k(k, intList))

[4, 1, 6]
